In [19]:
import iris
import Utils.iris_utils as iris_utils
import pandas as pd
import numpy as np
import os
import logging
import esmvalcore.preprocessor
import glob
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import xarray as xr
from xmip.preprocessing import rename_cmip6
import scipy.stats as stats



In [62]:

#exps = ["ssp370", "ssp126", "ssp245"]
exps = ["historical"]
#exps = ['ssp245']
#exps = ['ssp126']
dirs = []
var_path = "Amon/tas"
for experiment in exps:
    if experiment == "historical":
        exp_set = "CMIP"
    else:
        exp_set = "ScenarioMIP"
    for x in glob.glob('/badc/cmip6/data/CMIP6/{es}/*/*/{e}/*/{v}/*/latest/'.format(es=exp_set, e=experiment, v=var_path)):
        dirs.append(x)
dirs.reverse()
print(len(dirs))

394


In [57]:
#AA_years = ["2015", "2065"]
AA_years = ["1965", "2015"]
trend_period = str(AA_years[0] + '_' + AA_years[1])

def preprocess_multi_ens(folder, arctic_cut_off=66, wintertime=False):
    
    """ makes a df contining global mean and {global except arctic} mean
        temp by year, for the first ensemble member of each model """
    
    outpath = 'Outputs/mean_AAs/AA_{M}_{Exp}_{Ens}.csv'.format(
                M=folder.split('/')[7], Exp=folder.split('/')[8], Ens=folder.split('/')[9])
    
    #if os.path.exists(outpath):
    #    return
    #else:                                                              
    try:
        data = rename_cmip6(xr.open_mfdataset(folder + "*.nc", use_cftime=True))
        winter_months = [12,1,2]
        winter_mask = data.time.dt.month.isin(winter_months)
        name = str(folder.split('/')[7] + '_' + folder.split('/')[8] + '_' + folder.split('/')[9])
        
        at_data = data['tas']#.isel(member_id=0)
        if wintertime:
            at_data = at_data[winter_mask]
        at_data = at_data.sel(time=slice(AA_years[0], AA_years[1]))
    
        jan_mask = at_data.time.dt.month.isin([1]) 
        years = at_data.time.dt.year[jan_mask].compute()
        
        # month_length = data.time.dt.days_in_month
        # weights = ( month_length.groupby("time.season") / month_length.groupby("time.season").mean())
        world_annual = (at_data).groupby("time.year").mean(dim="time")
        arctic_annual = world_annual.sel(y=slice(arctic_cut_off, 90))
        
        world_w = world_annual.weighted(weights=np.cos(np.deg2rad(world_annual.y)))
        arctic_w = arctic_annual.weighted(weights=np.cos(np.deg2rad(arctic_annual.y)))
    
        arctic_temps = arctic_w.mean(("x","y")).compute().values
        world_temps = world_w.mean(("x","y")).compute().values
        
        arctic_trend = stats.linregress(years, arctic_temps).slope
        world_trend = stats.linregress(years, world_temps).slope
        AA = arctic_trend/world_trend
        
        print(arctic_trend, world_trend, AA)
    
        if wintertime:
            months=winter_months
        else:
            months='all'
    
        Model = folder.split('/')[7]
        Experiment = folder.split('/')[8]
        Ensemble_member = folder.split('/')[9]
        
        df = pd.DataFrame(columns=['arctic_trend', 'world_trend', 'AA', 
                                   'trend_period', 'Model', 'Experiment',
                                   'Ensemble_member', 'Months'])
        df.loc[0] = [arctic_trend, world_trend, AA,
                     trend_period, Model, Experiment,
                     Ensemble_member, months]
    
        df.to_csv(outpath) 
    except:
        print(name)

In [58]:
preprocess_multi_ens(dirs[0])

0.04689091942257021 0.02277407018668245 2.0589608725273236


In [64]:
for dir in tqdm(dirs):
    preprocess_multi_ens(dir)

  0%|          | 1/394 [00:04<27:58,  4.27s/it]

0.07151793400412708 0.020468574490593396 3.4940358957090094


  1%|          | 2/394 [01:00<3:48:21, 34.95s/it]

0.05979399490125664 0.02223268953169886 2.6894629557077985


  1%|          | 3/394 [01:45<4:17:51, 39.57s/it]

...

0.03391654186466088 0.015955748476769636 2.1256628552424726


 36%|███▌      | 142/394 [21:13<3:52:18, 55.31s/it]

0.020926496087119335 0.014806843177462104 1.4132989615890659


 36%|███▌      | 142/394 [22:01<39:05,  9.31s/it]  


UnboundLocalError: local variable 'name' referenced before assignment